In [1]:
import xgboost as xgb


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#Loading Library
import pandas as pd 
import numpy as np
from datetime import timedelta,datetime
from load_data import load_train,load_test
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

#train読み込み
train = load_train()

x_train = train.drop(["date","unit_sales"],axis=1)
y_train = train.unit_sales

#xgboostのために型変換
fac = x_train.columns

for feat in fac:
    x_train[feat] = pd.factorize(x_train[feat],sort = True)[0]
    
x_train = np.array(x_train)

#xgboostによりモデルフィッティング
reg = xgb.XGBRegressor()

reg_cv = GridSearchCV(reg,{"max_depth":[2,4,6],"n_estimators":[50,100,200]},verbose = 1)
reg_cv.fit(x_train,y_train)

reg = xgb.XGBRegressor(**reg_cv.best_params_)



reg.fit(x_train, y_train)


#test読み込み
test = load_test()

#モデルに当てはめる
pred_test = reg.predict(test)
submission = pd.DataFrame({"id":test_id,"unit_sales":pd.Series(pred_test)})
submission.to_csv("sub3.csv",index=False)


Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed: 201.1min finished


NameError: name 'train' is not defined

In [4]:

#test読み込み
test = load_test()

#モデルに当てはめる
pred_test = reg.predict(test)
submission = pd.DataFrame({"id":test_id,"unit_sales":pd.Series(pred_test)})
submission.to_csv("sub3.csv",index=False)

NameError: name 'train' is not defined

In [106]:
#Loading_test_data
dtypes = {'id':'uint32', 'item_nbr':'int32', 'store_nbr':'uint8', 'unit_sales':'float32',"onpromotion":"bool"}
test = pd.read_csv("test.csv",dtype = dtypes,parse_dates =["date"])
test_id = test["id"]
test['dow'] = test['date'].dt.dayofweek

#Loading mean_sales
mean_sales = pd.read_csv("mean_sales.csv")
mean_sales.drop(["mawk","mais226","madw","Unnamed: 0"],axis=1,inplace=True)
test = pd.merge(test,mean_sales,how="outer",on=["item_nbr","store_nbr","dow"])
test.drop(["item_nbr","store_nbr"],axis=1,inplace=True)
test["mais"] = test["mais"].fillna(test["mais"].mean())

#Loading_test_oil
oil = pd.read_csv("oil.csv",parse_dates=["date"],dtype={"dcoilwtico":"float32"},skiprows=range(1,1045))
test_oil = oil[datetime(2017,8,15) < oil.date]
oil_date = pd.date_range("8/16/2017","31/8/2017")
oil_date = pd.DataFrame(oil_date)

oil_date.columns= ["date"]

test_oil2 = pd.merge(test_oil,oil_date,how="outer",on="date")
test_oil2["dcoilwtico"].fillna(test_oil["dcoilwtico"].mean(),inplace=True)

test = pd.merge(test,test_oil2,how="outer",on="date")

test["payday"] = test["date"].apply(lambda s:True if s.day == 31else False)


test["holiday"] = test["date"].dt.day.apply(lambda s:True if s == 24 else False)

test = test.dropna(subset = ["date"])

test["month"] = test["date"].dt.month
test["month"] = test["month"].astype(np.int8)
test["dow"] = test["dow"].astype(np.int8)

del(test["id"])
del(test["date"])
del(oil)
del(test_oil)
del(test_oil2)

test.loc[test['onpromotion'] == True, 'mais'] = test.loc[test['onpromotion'] == True, 'mais']*1.5
test.onpromotion = test.onpromotion.astype("bool")
test = test.ix[:,["onpromotion","payday","dow","month","mais","dcoilwtico","holiday"]]

test = np.array(test)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:46: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [1]:
reg

NameError: name 'reg' is not defined

In [87]:
test

,onpromotion,dow,mais,dcoilwtico,payday,holiday,month
0,False,2,0.862129,46.799999,False,False,8
1,False,2,1.022292,46.799999,False,False,8
2,False,2,1.532617,46.799999,False,False,8
3,False,2,1.110800,46.799999,False,False,8
4,False,2,1.314988,46.799999,False,False,8
5,False,2,1.842727,46.799999,False,False,8
6,False,2,2.284627,46.799999,False,False,8
7,False,2,1.532617,46.799999,False,False,8
8,False,2,1.051670,46.799999,False,False,8
9,False,2,0.808954,46.799999,False,False,8


In [91]:
len(train.columns)

9

In [92]:
len(test.columns)


7

AttributeError: 'numpy.ndarray' object has no attribute 'onpromotion'

In [94]:
test.head()

,onpromotion,payday,dow,month,mais,dcoilwtico,holiday
0,False,False,2,8,0.862129,46.799999,False
1,False,False,2,8,1.022292,46.799999,False
2,False,False,2,8,1.532617,46.799999,False
3,False,False,2,8,1.110800,46.799999,False
4,False,False,2,8,1.314988,46.799999,False


In [108]:
#モデルに当てはめる
pred_test = reg.predict(test)
submission = pd.DataFrame({"id":test_id,"unit_sales":pd.Series(pred_test)})
submission.to_csv("sub3.csv",index=False)

In [99]:
submission.head()

,id,unit_sales
0,125497040,0.74089
1,125497041,0.74089
2,125497042,0.74089
3,125497043,0.74089
4,125497044,0.74089


In [32]:
test.dtypes

mais           float64
dow              int64
payday            bool
holiday           bool
payday            bool
dcoilwtico     float32
onpromotion       bool
dtype: object